In [2]:
import tensorflow as tf


In [3]:
class Constants():
    PAD_VOCAB = '<PAD>'
    UNK_VOCAB = '<UNK>'
    SOS_VOCAB = '<SOS>'
    EOS_VOCAB = '<EOS>'

In [4]:
import numpy as np
from typing import List
class VocabDict:
    def __init__(self, vocab_dict) -> None:
        self.vocab_dict = vocab_dict

    def index_to_vocab(self, index: int) -> str:
        for k, v in self.vocab_dict.items():
            if (v == index):
                return k
            continue

    def vocab_to_index(self, vocab: str) -> int:
        return self.vocab_dict[vocab]

    def list_of_index_to_vocab(self, list_of_index: List[int]):
        return [self.index_to_vocab(i) for i in list_of_index]

    def list_of_vocab_to_index(self, list_of_vocab: List[str]):
        return [self.vocab_to_index(v) for v in list_of_vocab]

    def vocab_size(self) -> int:
        '''
        Include <START>, <END> and <PAD> tokens. So, if you the actual number of activities,
        you have to minus 3.
        '''
        return len(self.vocab_dict)
    
    def padding_index(self):
        return self.vocab_to_index(Constants.PAD_VOCAB)

    def tranform_to_input_data_from_seq_idx_with_caseid(self, seq_list: List[List[int]], caseids: List[str] = None):
        '''
        Calculate the lengths for reach trace, so we can use padding.
        '''
        
        seq_lens = np.array([len(s)for s in seq_list])
        sorted_len_index = np.flip(np.argsort(seq_lens))
        sorted_seq_lens = [seq_lens[idx] for idx in sorted_len_index]
        sorted_seq_list = [tf.constant(seq_list[idx])
                           for idx in sorted_len_index]

        if (caseids):
            sorted_caseids = [caseids[i] for i in sorted_len_index]
        else:
            sorted_caseids = None

        return sorted_caseids, tf.keras.preprocessing.sequence.pad_sequences(sorted_seq_list, padding='post', value=0), tf.constant(sorted_seq_lens)

    def __len__(self):
        return self.vocab_size()


In [38]:
import pandas as pd

<function tensorflow.python.keras.backend.in_train_phase(x, alt, training=None)>

In [6]:
df = pd.read_pickle("./datasets/preprocessed/BPI_Challenge_2012/AOW/df.pickle")

In [70]:
temp = [ r.to_dict() for idx, r in df.iterrows()]

In [77]:
trace_arrays = [np.array(t)for t  in list(df['trace'])]

In [78]:
ds = tf.data.Dataset.from_tensor_slices((list(df["caseid"]), trace_arrays))

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [39]:
example =  iter(ds).next()

In [58]:
df[df['caseid'] == str(int(example.numpy()))]  # Find the correspond case id.

,trace,caseid
0,"[2, 12, 9, 10, 27, 28, 3, 17, 8, 15, 19, 33, 2...",173688


In [64]:
ds = tf.data.Dataset.from_tensor_slices(list(range(len(df))))

In [97]:
ds = tf.data.Dataset.range(len(df))

In [98]:
ds = ds.shuffle(20)
ds = ds.batch(8)

In [110]:
batch_idx = list(ds.as_numpy_iterator())[0]
batch_df = df.iloc[batch_idx]

In [118]:
caseids = list(batch_df["caseid"])

In [1]:
caseids

NameError: name 'caseids' is not defined

In [2]:
from Parameters import TrainingParameters, EnviromentParameters

In [3]:
EnviromentParameters.BPI2020Dataset.preprocessed_foldr_path

'./datasets/preprocessed/BPI_Challenge_2012'

In [28]:
train_param = TrainingParameters()
xes_dataset = XESDataset(file_path=EnviromentParameters.BPI2020Dataset.file_path,preprocessed_folder_path=EnviromentParameters.BPI2020Dataset.preprocessed_foldr_path,
preprocessed_df_type= EnviromentParameters.BPI2020Dataset.preprocessed_df_type,
include_types = train_param.bpi2012.BPI2012_include_types
)


| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012/OW 


In [35]:
ds = dataset.get_index_ds()
ds = ds.shuffle(3).batch(32)
batch_idx = list(ds.as_numpy_iterator())[0]
caseids, input_data, target = xes_dataset.collate_fn(batch_idx)

In [18]:
dataset.collate_fn(batch_idx)[1].shape

(32, 101)

In [120]:
batch_traces = list(batch_df["trace"])

In [122]:
data_traces = [ t[:-1] for t in batch_traces]

In [126]:
target_traces = [t[1:] for t in batch_traces]

In [128]:
padded_data_traces = tf.keras.preprocessing.sequence.pad_sequences(data_traces, padding='post', value=0)

In [19]:
padded_target_traces = tf.keras.preprocessing.sequence.pad_sequences(target_traces, padding='post', value=0)

NameError: name 'tf' is not defined

In [23]:
lstm_hidden = 16
lstm = tf.keras.models.Sequential(
    [tf.keras.layers.LSTM(
                lstm_hidden,
                return_sequences = True,
                return_state=True,
                stateful=True
    ),
    tf.keras.layers.LSTM(
                lstm_hidden,
                return_sequences = True,
                return_state=True,
                stateful=True
    )]
)

In [25]:
out_net = tf.keras.models.Sequential(
    [
         tf.keras.layers.BatchNormalization(),
         tf.keras.layers.LeakyReLU(),
         tf.keras.layers.Dropout(.2),
         tf.keras.layers.Dense(30),
    ]
)

In [31]:
class BaselineLSTM(tf.keras.Model):
    def __init__(self, vocab: VocabDict,  embedding_dim: int, lstm_hidden: int, dropout: float):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(
            input_dim = len(vocab),
            output_dim = embedding_dim,
            mask_zero=True
        )
        self.lstm =  tf.keras.layers.LSTM(
                lstm_hidden,
                return_sequences = True,
                return_state=True,
                stateful=True
        )
        self.out_net = tf.keras.models.Sequential(
            [
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.LeakyReLU(),
                tf.keras.layers.Dropout(dropout),
                tf.keras.layers.Dense(len(vocab)),
            ]
        )


    def call(self, inputs, state = None):
        if not state is None:
            self.lstm.reset_states(state)

        out = self.emb(inputs)
        out, h_out, c_out = self.lstm(out)
        out = self.out_net(out)
        out = tf.nn.softmax(out, axis= -1)

        return out, (h_out, c_out)


In [36]:
baseline_lstm = BaselineLSTM(
    xes_dataset.vocab, 
    embedding_dim= train_param.baselineLSTMModelParameters.embedding_dim,
    lstm_hidden= train_param.baselineLSTMModelParameters.lstm_hidden,
    dropout=train_param.baselineLSTMModelParameters.dropout
)

In [37]:
baseline_lstm(input_data)

(<tf.Tensor: shape=(32, 101, 29), dtype=float32, numpy=
 array([[[0.0344136 , 0.03450113, 0.03452059, ..., 0.03445655,
          0.0345066 , 0.03448131],
         [0.03446138, 0.03447157, 0.03440817, ..., 0.03442244,
          0.03440362, 0.03454718],
         [0.03454544, 0.0343913 , 0.03460056, ..., 0.03450439,
          0.03428044, 0.03433595],
         ...,
         [0.03484194, 0.03413647, 0.03464504, ..., 0.0344089 ,
          0.03451387, 0.03425217],
         [0.03484194, 0.03413647, 0.03464504, ..., 0.0344089 ,
          0.03451387, 0.03425217],
         [0.03484194, 0.03413647, 0.03464504, ..., 0.0344089 ,
          0.03451387, 0.03425217]],
 
        [[0.03441359, 0.03450113, 0.03452059, ..., 0.03445655,
          0.03450659, 0.0344813 ],
         [0.03446138, 0.03447157, 0.03440817, ..., 0.03442244,
          0.03440362, 0.03454718],
         [0.03454544, 0.0343913 , 0.03460056, ..., 0.03450439,
          0.03428044, 0.03433595],
         ...,
         [0.03450479, 0.0344244

In [33]:
lstm()

In [134]:
emb = tf.keras.layers.Embedding(
            input_dim = vocab_len,
            output_dim = 32,
            mask_zero=True
        )

In [182]:
lstm = tf.keras.layers.LSTM(
                64,
                return_sequences = True,
                return_state=True,
                stateful=True
        )

In [183]:
input_data = padded_data_traces

In [184]:
emb_out = emb(input_data)

In [185]:
mask = emb.compute_mask(input_data)

In [186]:
lstm_out, h_, c_ = lstm(emb_out, mask= mask)

In [188]:
lstm.reset_states((h_, c_))

In [200]:
input_data.shape

(8, 60)

In [199]:
lstm.states[0].shape

TensorShape([8, 64])

In [1]:
import pm4py

In [159]:
bn = tf.keras.layers.BatchNormalization()
re = tf.keras.layers.LeakyReLU()
drop = tf.keras.layers.Dropout(.2)
dense = tf.keras.layers.Dense(vocab_len)
softmax_out = tf.nn.softmax(dense_out, axis= -1)

In [160]:
bn_out = bn(lstm_out)

In [161]:
re = tf.keras.layers.LeakyReLU()

In [162]:
re_out = re(bn_out)

In [163]:
drop = tf.keras.layers.Dropout(.2)

In [164]:
drop_out = drop(re_out)

In [213]:
dense = tf.keras.layers.Dense(vocab_len)

In [214]:
dense_out = dense(drop_out)

In [215]:
softmax_out = tf.nn.softmax(dense_out, axis= -1)

In [216]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [217]:
cce = tf.keras.losses.CategoricalCrossentropy()

In [224]:
loss = cce(softmax_out, tf.one_hot(padded_target_traces, depth=vocab_len))

In [223]:
sparse_cce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=padded_target_traces, y_pred=softmax_out)

In [225]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=15.7052965>

In [226]:
sparse_cce_loss

<tf.Tensor: shape=(8, 60), dtype=float32, numpy=
array([[3.6641452, 3.6601377, 3.6568236, 3.6619198, 3.6659055, 3.659903 ,
        3.6613874, 3.6573622, 3.6578019, 3.662001 , 3.665196 , 3.6573966,
        3.6685224, 3.6639829, 3.6655066, 3.6655397, 3.6615639, 3.6660523,
        3.6567807, 3.6647983, 3.6568944, 3.6568944, 3.6568944, 3.6568942,
        3.6568942, 3.6568944, 3.6568942, 3.6568942, 3.6568944, 3.6568944,
        3.6568944, 3.6568942, 3.6568942, 3.6568944, 3.6568942, 3.6568942,
        3.6568944, 3.6568944, 3.6568944, 3.6568942, 3.6568942, 3.6568944,
        3.6568942, 3.6568942, 3.6568944, 3.6568944, 3.6568944, 3.6568942,
        3.6568942, 3.6568944, 3.6568942, 3.6568942, 3.6568944, 3.6568944,
        3.6568944, 3.6568942, 3.6568942, 3.6568944, 3.6568942, 3.6568942],
       [3.6641452, 3.6601377, 3.6568236, 3.6619196, 3.6659055, 3.6599033,
        3.6613874, 3.6573622, 3.6643739, 3.6623595, 3.6692684, 3.666542 ,
        3.662608 , 3.6712987, 3.6649058, 3.6730618, 3.6730618,

In [227]:
tf.reduce_mean(sparse_cce)

<tf.Tensor: shape=(), dtype=float32, numpy=3.664769>

In [52]:
type(list(df["caseid"])[0])

str

In [59]:
tf.data.Dataset.from_tensor_slices({"caseid": [1,2,3], "trace": [[1.3],[3,4,6],[1]]})

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [ ]:
from Utils.PrintUtils import print_big
from CustomExceptions import NotSupportedError
import pandas as pd
from torch.utils.data import Dataset
import pm4py
from datetime import timedelta
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from Utils.Constants import Constants
from Utils.FileUtils import file_exists
import json
import os
from Parameters.Enums import PreprocessedDfType, ActivityType

class XESDataset(Dataset):
    pickle_df_file_name = "df.pickle"
    vocab_dict_file_name = "vocab_dict.json"

    def __init__(self, file_path: str, preprocessed_folder_path: str, preprocessed_df_type: PreprocessedDfType, device: torch.device = torch.device("cpu"), include_types: list[ActivityType] = None ) -> None:
        super().__init__()
        self.device = device
        self.file_path = file_path
        self.preprocessed_folder_path = os.path.join(
            preprocessed_folder_path, XESDataset.get_type_folder_name(include_types))
        self.preprocessed_df_type = preprocessed_df_type

        if (not preprocessed_folder_path is None) and self.preprocessed_data_exist(self.preprocessed_folder_path, self.preprocessed_df_type):
            self.load_preprocessed_data()
        else:
            self.__initialise_data(
                file_path=file_path, include_types=include_types)

            if not preprocessed_folder_path is None:
                self.save_preprocessed_data()

    def __initialise_data(self, file_path: str, include_types: list[ActivityType]) -> None:
        '''
        run this function if the preprocessed data doesn't exist.
        [file_path]: path of `BPI_Challenge_2012.xes` 
        [include_types]: what types of activity you want to load.
        '''
        ############ load xes file and extract needed information ############
        log = pm4py.read_xes(file_path)
        flattern_log: list[dict[str, any]] = ([{**event,
                                                'caseid': trace.attributes['concept:name']}
                                               for trace in log for event in trace])
        df = pd.DataFrame(flattern_log)

        if not (include_types is None):
            df = df[[any(bool_set) for bool_set in zip(
                *([df["concept:name"].str.startswith(a.value) for a in include_types]))]]

        df["name_and_transition"] = df["concept:name"] + \
            "_" + df["lifecycle:transition"]
        df = df[['time:timestamp', 'name_and_transition', "caseid"]]

        ############ Append starting and ending time for each trace ############
        newData = list() 
        for case, group in df.groupby('caseid'):
            group.sort_values("time:timestamp", ascending=True, inplace=True)
            strating_time = group.iloc[0]["time:timestamp"] - \
                timedelta(microseconds=1)
            ending_time = group.iloc[-1]["time:timestamp"] + \
                timedelta(microseconds=1)
            traces = group.to_dict('records')

            # Add start and end tags.
            traces.insert(
                0, {"caseid": case, "time:timestamp": strating_time, "name_and_transition": Constants.SOS_VOCAB})
            traces.append(
                {"caseid": case, "time:timestamp": ending_time, "name_and_transition": Constants.EOS_VOCAB})
            newData.extend(traces)

        df = pd.DataFrame(newData)
        df['name_and_transition'] = df['name_and_transition'].astype(
            'category')

        ############ generate vocabulary dictionary ############
        vocab_dict: dict[str, int] = {}
        for i, cat in enumerate(df['name_and_transition'].cat.categories):
            # plus one, since we want to remain "0" for "<PAD>"
            vocab_dict[cat] = i+1
        vocab_dict[Constants.PAD_VOCAB] = 0

        ############ Create new index categorial column ############
        df['cat'] = df['name_and_transition'].apply(lambda c: vocab_dict[c])

        ############ Create the df only consisted of trace and caseid ############
        final_df_data: list[dict[str, any]] = []
        for caseid, group in df.groupby('caseid'):
            final_df_data.append({
                "trace": list(group['cat']),
                "caseid": caseid
            })

        ############ store data in instance ############    
        self.df: pd.DataFrame = pd.DataFrame(final_df_data)
        self.df.sort_values("caseid", inplace=True)
        self.vocab = VocabDict(vocab_dict)

    def longest_trace_len(self) -> int:
        return self.df.trace.map(len).max()

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, index: int) -> pd.Series:
        return self.df.iloc[index]

    @staticmethod
    def get_type_folder_name(include_types: list[ActivityType] = None):
        if include_types is None:
            return "All"

        return "".join(
            sorted([a.value for a in include_types], key=str.lower))

    @staticmethod
    def get_file_name_from_preprocessed_df_type(preprocessed_df_type: PreprocessedDfType):
        if preprocessed_df_type == PreprocessedDfType.Pickle:
            return XESDataset.pickle_df_file_name
        else:
            raise NotSupportedError(
                "Not supported saving format for preprocessed data")

    @staticmethod
    def preprocessed_data_exist(preprocessed_folder_path: str, preprocessed_df_type: PreprocessedDfType, ):
        file_name = XESDataset.get_file_name_from_preprocessed_df_type(
            preprocessed_df_type)
        df_path = os.path.join(preprocessed_folder_path,  file_name)
        vocab_dict_path = os.path.join(
            preprocessed_folder_path, XESDataset.vocab_dict_file_name)
        return file_exists(df_path) and file_exists(vocab_dict_path)

    def store_df(self, preprocessed_folder_path: str, preprocessed_df_type: PreprocessedDfType):
        os.makedirs(preprocessed_folder_path, exist_ok=True)
        file_name = XESDataset.get_file_name_from_preprocessed_df_type(
            preprocessed_df_type)
        df_path = os.path.join(preprocessed_folder_path, file_name)
        if(preprocessed_df_type == PreprocessedDfType.Pickle):
            self.store_df_in_pickle(df_path)
        else:
            raise NotSupportedError(
                "Not supported saving format for preprocessed data")

    def load_df(self, preprocessed_folder_path: str, preprocessed_df_type: PreprocessedDfType):
        file_name = XESDataset.get_file_name_from_preprocessed_df_type(
            preprocessed_df_type)
        df_path = os.path.join(preprocessed_folder_path, file_name)
        if(preprocessed_df_type == PreprocessedDfType.Pickle):
            self.load_df_from_pickle(df_path)
        else:
            raise NotSupportedError(
                "Not supported loading format for preprocessed data")

    def store_df_in_pickle(self, path):
        self.df.to_pickle(path)

    def load_df_from_pickle(self, path):
        self.df = pd.read_pickle(path)

    def save_preprocessed_data(self):
        if self.preprocessed_folder_path is None:
            raise Error("Preprocessed folder path can't be None")

        ############ Store df ############
        self.store_df(self.preprocessed_folder_path,
                      self.preprocessed_df_type)

        ############ Store vocab_dict ############
        vocab_dict_path = os.path.join(
            self.preprocessed_folder_path, XESDataset.vocab_dict_file_name)
        with open(vocab_dict_path, 'w') as output_file:
            json.dump(self.vocab.vocab_dict, output_file, indent='\t')

        print_big(
            "Preprocessed data saved successfully"
        )

    def load_preprocessed_data(self):
        if self.preprocessed_folder_path is None:
            raise Error("Preprocessed folder path can't be None")

        ############ Load df ############
        self.load_df(self.preprocessed_folder_path, self.preprocessed_df_type)

        ############ load vocab_dict ############
        vocab_dict_path = os.path.join(
            self.preprocessed_folder_path, XESDataset.vocab_dict_file_name)
        with open(vocab_dict_path, 'r') as output_file:
            vocab_dict = json.load(output_file)
            self.vocab = VocabDict(vocab_dict)

        print_big(
            "Preprocessed data loaded successfully: %s" % (self.preprocessed_folder_path)
        )

    def collate_fn(self, data: list[pd.Series]) -> tuple[np.ndarray, torch.Tensor, torch.Tensor, np.ndarray]:
        caseid_list, seq_list = zip(
            *[(d["caseid"], torch.tensor(d["trace"])) for d in data])
        caseid_list = list(caseid_list)
        seq_list = list(seq_list)

        ############ Get sorting index ############
        seq_lens_before_splitting = np.array([len(s)for s in seq_list])
        sorted_len_index = np.flip(np.argsort(seq_lens_before_splitting))

        ############ Sort caseids and traces ############
        sorted_seq_list = [seq_list[idx]
                           for idx in sorted_len_index]
        sorted_case_id = np.array(caseid_list)[sorted_len_index]

        ############ Build training and test seq ############
        # this should remove all the EOS to form a training set
        data_seq_list = [li[:-1] for li in sorted_seq_list]
        
        # this should remove all the SOS to form a testing set
        target_seq_list = [li[1:] for li in sorted_seq_list]

        ############ Get lengths ############
        data_seq_length = [len(l) for l in data_seq_list]

        ############ Pad input and target ############
        padded_data = pad_sequence(
            data_seq_list, batch_first=True,  padding_value=0)
        padded_target = pad_sequence(
            target_seq_list, batch_first=True, padding_value=0)

        return sorted_case_id, padded_data.to(self.device), torch.tensor(data_seq_length).to(self.device),padded_target.to(self.device)

    def get_sampler_from_df(self, df, seed):
        return None

    def get_train_shuffle(self):
        return True
